### Libraries

In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
from scipy.optimize import minimize

### Reading and cleaning the data

In [49]:
data_ccc = pd.read_csv('financial/ccc_d.csv')
data_dino = pd.read_csv('financial/dino.csv')
data_kghm = pd.read_csv('financial/kgh_d.csv')
data_orlen = pd.read_csv('financial/pkn_d.csv')
data_pko = pd.read_csv('financial/pko_d.csv')

In [50]:
data_ccc = data_ccc.sort_values("Data",ascending=True)
data_ccc = data_ccc.rename(columns={'Data':"Date","Zamkniecie":"Close"})
data_ccc = data_ccc.loc[:,data_ccc.columns.isin(["Date","Close"])]

data_dino = data_dino.sort_values("Data",ascending=True)
data_dino = data_dino.rename(columns={"Data":"Date","Zamkniecie":"Close"})
data_dino = data_dino.loc[:,data_dino.columns.isin(["Date","Close"])]

data_kghm = data_kghm.sort_values("Data",ascending=True)
data_kghm = data_kghm.rename(columns = {"Data":"Date","Zamkniecie":"Close"})
data_kghm = data_kghm.loc[:,data_kghm.columns.isin(["Date","Close"])]

data_orlen = data_orlen.sort_values("Data",ascending=True)
data_orlen = data_orlen.rename(columns = {"Data":"Date","Zamkniecie":"Close"})
data_orlen = data_orlen.loc[:,data_orlen.columns.isin(["Date","Close"])]

data_pko = data_pko.sort_values("Data",ascending=True)
data_pko = data_pko.rename(columns = {"Data":"Date","Zamkniecie":"Close"})
data_pko = data_pko.loc[:,data_pko.columns.isin(["Date","Close"])]

In [54]:
m1 = pd.merge(data_ccc,data_pko,how="inner", on = "Date")
m1.columns = ["Date","close_ccc","close_pko"]

m2 = pd.merge(m1,data_dino,how="inner", on = "Date")
m2.columns = ["Date","close_ccc","close_pko","close_dino"]

m3 = pd.merge(m2, data_kghm,how="inner", on = "Date")
m3.columns = ["Date","close_ccc","close_pko","close_dino","close_kghm"]

df = pd.merge(m3, data_orlen,how="inner", on = "Date")
df.columns = ["Date","close_ccc","close_pko","close_dino","close_kghm","close_orlen"]

In [55]:
df

,Date,close_ccc,close_pko,close_dino,close_kghm,close_orlen
0,2022-07-25,44.02,26.549936,331.2,102.999985,76.440000
1,2022-07-26,42.31,25.749984,335.4,104.100832,74.400025
2,2022-07-27,41.30,25.500019,350.7,101.549637,74.239959
3,2022-07-28,42.13,25.630018,358.1,109.050062,74.980017
4,2022-07-29,43.40,26.190040,361.5,115.449436,75.520047
...,...,...,...,...,...,...
497,2024-07-17,122.30,58.880000,396.9,145.300000,64.640000
498,2024-07-18,124.90,61.000000,407.3,142.250000,65.170000
499,2024-07-19,122.00,60.160000,402.8,137.400000,65.180000
500,2024-07-22,123.90,60.800000,398.3,133.750000,64.510000


### Calculating log return rates

In [57]:
df['ccc_log_return'] = np.log(df['close_ccc'] / df['close_ccc'].shift(1))
df['pko_log_return'] = np.log(df['close_pko'] / df['close_pko'].shift(1))
df['dino_log_return'] = np.log(df['close_dino'] / df['close_dino'].shift(1))
df['orlen_log_return'] = np.log(df['close_orlen'] / df['close_orlen'].shift(1))

In [58]:
df

,Date,close_ccc,close_pko,close_dino,close_kghm,close_orlen,ccc_log_return,pko_log_return,dino_log_return,orlen_log_return
0,2022-07-25,44.02,26.549936,331.2,102.999985,76.440000,NaN,NaN,NaN,NaN
1,2022-07-26,42.31,25.749984,335.4,104.100832,74.400025,-0.039621,-0.030593,0.012601,-0.027050
2,2022-07-27,41.30,25.500019,350.7,101.549637,74.239959,-0.024161,-0.009755,0.044607,-0.002154
3,2022-07-28,42.13,25.630018,358.1,109.050062,74.980017,0.019898,0.005085,0.020881,0.009919
4,2022-07-29,43.40,26.190040,361.5,115.449436,75.520047,0.029699,0.021615,0.009450,0.007177
...,...,...,...,...,...,...,...,...,...,...
497,2024-07-17,122.30,58.880000,396.9,145.300000,64.640000,0.034945,-0.013495,0.004545,-0.005554
498,2024-07-18,124.90,61.000000,407.3,142.250000,65.170000,0.021036,0.035372,0.025866,0.008166
499,2024-07-19,122.00,60.160000,402.8,137.400000,65.180000,-0.023492,-0.013866,-0.011110,0.000153
500,2024-07-22,123.90,60.800000,398.3,133.750000,64.510000,0.015454,0.010582,-0.011235,-0.010332


In [73]:
for col in ['ccc_log_return','pko_log_return','dino_log_return','orlen_log_return']:
    print(f'{col}:\n\nSum of log return: {np.sum(df[col].iloc[1:])}\nAverage log return: {np.mean(df[col].iloc[1:])}\nStandard deviation of log return: {statistics.stdev(df[col].iloc[1:])}\n')

ccc_log_return:

Sum of log return: 0.986887647105001
Average log return: 0.0019698356229640736
Standard deviation of log return: 0.030194668150274787

pko_log_return:

Sum of log return: 0.8052668826039173
Average log return: 0.0016073191269539268
Standard deviation of log return: 0.01984893471022052

dino_log_return:

Sum of log return: 0.1575094221657778
Average log return: 0.00031439006420314926
Standard deviation of log return: 0.020640346237227315

orlen_log_return:

Sum of log return: -0.17793558187284886
Average log return: -0.0003551608420615746
Standard deviation of log return: 0.019930311284860728



In [83]:
log_return_cols = ['ccc_log_return','pko_log_return','dino_log_return','orlen_log_return']

# Calculate covariance matrix
cov_matrix = df[log_return_cols].cov()*252

In [84]:
cov_matrix

,ccc_log_return,pko_log_return,dino_log_return,orlen_log_return
ccc_log_return,0.229753,0.042642,0.032777,0.041342
pko_log_return,0.042642,0.099283,0.025920,0.044044
dino_log_return,0.032777,0.025920,0.107358,0.026546
orlen_log_return,0.041342,0.044044,0.026546,0.100099


### Portfolio standard deviation, expected return and sharpe ratio functions

In [85]:
def standard_deviation(weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

In [86]:
def expected_return(weights, log_returns):
    return np.sum(log_returns.mean()*weights)*252

In [87]:
def sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return (expected_return(weights, log_returns) - risk_free_rate) / standard_deviation(weights, cov_matrix)

In [100]:
log_returns = df.iloc[1:,-4:]
tickers = log_returns.columns

### Maximizing sharpe ratio

In [105]:
risk_free_rate = .02

def neg_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return -sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate)

constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
bounds = [(0, 0.4) for _ in range(len(tickers))]
initial_weights = np.array([1/len(tickers)]*len(tickers))

optimized_results = minimize(neg_sharpe_ratio, initial_weights, args=(log_returns, cov_matrix, risk_free_rate), method='SLSQP', constraints=constraints, bounds=bounds)

### Optimal weights

In [106]:
optimal_weights = optimized_results.x

In [108]:
optimal_weights

array([0.4, 0.4, 0.2, 0. ])

In [109]:
log_returns

,ccc_log_return,pko_log_return,dino_log_return,orlen_log_return
1,-0.039621,-0.030593,0.012601,-0.027050
2,-0.024161,-0.009755,0.044607,-0.002154
3,0.019898,0.005085,0.020881,0.009919
4,0.029699,0.021615,0.009450,0.007177
5,0.013730,0.027489,0.012371,0.007651
...,...,...,...,...
497,0.034945,-0.013495,0.004545,-0.005554
498,0.021036,0.035372,0.025866,0.008166
499,-0.023492,-0.013866,-0.011110,0.000153
500,0.015454,0.010582,-0.011235,-0.010332


In [107]:
print("Optimal Weights:")
for ticker, weight in zip(tickers, optimal_weights):
    print(f"{ticker}: {weight:.4f}")

optimal_portfolio_return = expected_return(optimal_weights, log_returns)
optimal_portfolio_volatility = standard_deviation(optimal_weights, cov_matrix)
optimal_sharpe_ratio = sharpe_ratio(optimal_weights, log_returns, cov_matrix, risk_free_rate)

print(f"Expected Annual Return: {optimal_portfolio_return:.4f}")
print(f"Expected Volatility: {optimal_portfolio_volatility:.4f}")
print(f"Sharpe Ratio: {optimal_sharpe_ratio:.4f}")

Optimal Weights:
ccc_log_return: 0.4000
pko_log_return: 0.4000
dino_log_return: 0.2000
orlen_log_return: 0.0000
Expected Annual Return: 0.3764
Expected Volatility: 0.2828
Sharpe Ratio: 1.2603
